In [9]:
import glob
import pandas as pd
import numpy as np

import json
from tqdm import tqdm
import os

import numpy as np
import socket

import pickle


In [10]:
# remove warnings

import warnings

warnings.filterwarnings("ignore")

In [11]:
version = "djia_stock_v1"
n_prediction = 10
output_version = f"{version}_n{n_prediction}"

location = "/home/ted/data"

data_dir = f"{location}/feature/{version}"
final_feature_dir = f"{location}/final_feature/{output_version}"


In [12]:
print(output_version)

djia_stock_v1_n10


# Reading data

In [13]:
# token_names = pickle.load(open(token_name, "rb"))

In [14]:
path_dict = {}

for filepath in glob.iglob(f"{data_dir}/*.pkl"):
    name = filepath.split("/")[-1]
    name = name.split(".")[0]
    path_dict[name] = filepath

In [15]:
# try read all into memory

dfs = []

first = True

for name, p in tqdm(path_dict.items()):

    df = pd.read_pickle(p)
    df = df.assign(stock_name=name.rstrip(".pkl"))
    
    # size we resample 24h, so the first few rows will be excluded
    
    min_t = df.index.get_level_values(0).min()
    df = df[df.index.get_level_values(0) > min_t + pd.to_timedelta("24h")]

    # drop features na
    if first:
        drop_sets = [x for x in df.columns if "target_future_ret" not in x or "std" not in x]
        first = False
        print(f"drop sets: {drop_sets}")

    df = df.dropna(subset=drop_sets, how="any")
    dfs.append(df)

df = pd.concat(dfs, 0)
df = df.set_index(["stock_name"], append=True)

100%|██████████| 30/30 [00:00<00:00, 228.70it/s]

drop sets: ['00_price_rising_rate0001', '00_target', '00_price_rising_rate0003', '00_price_rising_rate0006', '00_price_rising_rate0009', '00_price_rising_rate0030', '00_price_rising_rate0060', '00_price_rising_rate0090', '00_price_rising_rate0120', '01_vol_7', '01_vol_30', '01_vol_60', '01_vol_90', '01_vol_180', '01_vol_360', '02_trade_vol_1', '02_trade_vol_3', '02_trade_vol_7', '02_trade_vol_30', '02_trade_vol_60', '02_trade_vol_90', 'stock_name']


In [16]:
df.head()

,,00_price_rising_rate0001,00_target,00_price_rising_rate0003,00_price_rising_rate0006,00_price_rising_rate0009,00_price_rising_rate0030,00_price_rising_rate0060,00_price_rising_rate0090,00_price_rising_rate0120,01_vol_7,...,01_vol_60,01_vol_90,01_vol_180,01_vol_360,02_trade_vol_1,02_trade_vol_3,02_trade_vol_7,02_trade_vol_30,02_trade_vol_60,02_trade_vol_90
date,stock_name,,,,,,,,,,,,,,,,,,,,,
1981-08-19,INTC,1.016531,0.024389,1.000000,0.960940,0.946155,0.938932,0.788463,0.854168,0.878574,0.003596,...,0.016225,0.016393,0.017535,0.032499,3484800.0,13670400.0,39350400.0,204796800.0,541603200.0,870115200.0
1981-08-20,INTC,1.024389,-0.039683,1.032788,1.016128,1.000000,0.940299,0.768294,0.823529,0.875000,0.003596,...,0.016077,0.016612,0.017475,0.032493,5577600.0,13920000.0,36988800.0,200044800.0,532262400.0,867436800.0
1981-08-21,INTC,0.960317,-0.016528,1.000000,0.960317,0.960317,0.883211,0.728914,0.785713,0.858154,0.003025,...,0.015641,0.016884,0.017453,0.032511,8476800.0,17539200.0,41424000.0,201283200.0,526012800.0,867244800.0
1981-08-24,INTC,0.983472,-0.016805,0.967478,0.967478,0.929689,0.868613,0.725611,0.767742,0.856116,0.003733,...,0.015060,0.017183,0.017383,0.032555,6441600.0,20496000.0,39782400.0,204278400.0,505363200.0,864796800.0
1981-08-25,INTC,0.983195,-0.025643,0.928573,0.959019,0.943549,0.866667,0.726709,0.745225,0.866667,0.004159,...,0.014610,0.017508,0.017559,0.032611,13132800.0,28051200.0,47299200.0,207734400.0,507340800.0,840585600.0


In [17]:
df.tail()

,,00_price_rising_rate0001,00_target,00_price_rising_rate0003,00_price_rising_rate0006,00_price_rising_rate0009,00_price_rising_rate0030,00_price_rising_rate0060,00_price_rising_rate0090,00_price_rising_rate0120,01_vol_7,...,01_vol_60,01_vol_90,01_vol_180,01_vol_360,02_trade_vol_1,02_trade_vol_3,02_trade_vol_7,02_trade_vol_30,02_trade_vol_60,02_trade_vol_90
date,stock_name,,,,,,,,,,,,,,,,,,,,,
2022-08-24,CRM,1.022784,-0.033887,0.979540,0.949470,0.964012,1.101046,1.090309,0.950372,0.879170,5.870755,...,9.407252,10.298476,29.237392,39.151981,11438800.0,21698500.0,35853400.0,138261100.0,401663200.0,629483500.0
2022-08-25,CRM,0.966113,-0.049911,0.982653,0.925250,0.915846,1.079918,1.085310,0.929950,0.856657,6.118278,...,9.278567,10.191764,28.953219,39.243692,24025600.0,40541200.0,56043100.0,156190600.0,407574200.0,648609600.0
2022-08-26,CRM,0.950089,-0.030382,0.938807,0.879210,0.864807,0.987155,0.938434,0.863316,0.842022,7.675522,...,9.142480,10.167686,28.602810,39.369134,11074700.0,46539100.0,63936500.0,161227600.0,381611500.0,653563900.0
2022-08-29,CRM,0.969618,-0.003371,0.890006,0.871796,0.845034,0.954256,0.850372,0.860280,0.834080,9.160965,...,9.415311,10.134108,28.280299,39.510763,9329900.0,44430200.0,70474800.0,166739300.0,371710200.0,656878400.0
2022-08-30,CRM,0.996629,-0.022233,0.918119,0.902192,0.849489,0.918647,0.863501,0.900920,0.785932,9.133934,...,9.582505,10.179113,27.972748,39.648747,8079400.0,28484000.0,74208100.0,170445300.0,370565600.0,655838300.0


In [18]:
# checking token numbers
len(df.index.get_level_values(1).unique())

30

# Try normalization

In [19]:
def normalization(df):
    
    # columns set up
    target_cols = [x for x in df.columns if "target" in x]
    std_cols = [x for x in df.columns if "vol" in x]
    
    # select features for normalization
    df_target = df[target_cols]
    df_change_std = df[std_cols]
    
    df_no_size = df.drop(target_cols + std_cols, 1)

    # normalization
    mean = df_no_size.mean(level=0)
    std = df_no_size.std(level=0)

    df_no_size_norm = (df_no_size - mean) / std

    df_norm = (
        df_no_size_norm
        .join(df_target, how="left")
        .join(df_change_std, how="left")
    )

    assert df_no_size_norm.shape[0] == df_norm.shape[0]

    # remove row, where target is lost
    df_norm_dropna_target = df_norm.dropna(subset=[target_cols[0]], axis=0)
    return df_norm_dropna_target

In [20]:
# processing
from pprint import pprint

df_norm_dropna_target = normalization(df)

pprint(df_norm_dropna_target.columns.tolist())

# create folder for save
if not os.path.exists(final_feature_dir):
    os.makedirs(final_feature_dir)

df_norm_dropna_target.to_pickle(f"{final_feature_dir}/data.pkl")

['00_price_rising_rate0001',
 '00_price_rising_rate0003',
 '00_price_rising_rate0006',
 '00_price_rising_rate0009',
 '00_price_rising_rate0030',
 '00_price_rising_rate0060',
 '00_price_rising_rate0090',
 '00_price_rising_rate0120',
 '00_target',
 '01_vol_7',
 '01_vol_30',
 '01_vol_60',
 '01_vol_90',
 '01_vol_180',
 '01_vol_360',
 '02_trade_vol_1',
 '02_trade_vol_3',
 '02_trade_vol_7',
 '02_trade_vol_30',
 '02_trade_vol_60',
 '02_trade_vol_90']


In [21]:
df_norm_dropna_target.groupby(level=1)["00_price_rising_rate0001"].count() / 365

stock_name
AAPL    27.830137
AMGN    26.090411
AXP     33.731507
BA      35.410959
CAT     35.410959
CRM     11.561644
CSCO    21.471233
CVX     35.410959
DIS     35.410959
DOW      1.397260
GS      15.098630
HD      27.295890
HON     35.410959
IBM     35.410959
INTC    28.345205
JNJ     35.410959
JPM     28.345205
KO      35.410959
MCD     35.410959
MMM     35.410959
MRK     35.410959
MSFT    24.197260
NKE     27.852055
PG      35.410959
TRV     31.339726
UNH     25.164384
V        8.983562
VZ      25.791781
WBA     28.345205
WMT     33.567123
Name: 00_price_rising_rate0001, dtype: float64

In [22]:
df_norm_dropna_target

,,00_price_rising_rate0001,00_price_rising_rate0003,00_price_rising_rate0006,00_price_rising_rate0009,00_price_rising_rate0030,00_price_rising_rate0060,00_price_rising_rate0090,00_price_rising_rate0120,00_target,01_vol_7,...,01_vol_60,01_vol_90,01_vol_180,01_vol_360,02_trade_vol_1,02_trade_vol_3,02_trade_vol_7,02_trade_vol_30,02_trade_vol_60,02_trade_vol_90
date,stock_name,,,,,,,,,,,,,,,,,,,,,
1981-08-19,INTC,1.347883,1.055432,-0.783576,-1.414478,-0.698753,-2.112928,-1.032831,-0.970862,0.024389,0.003596,...,0.016225,0.016393,0.017535,0.032499,3484800.0,13670400.0,39350400.0,204796800.0,541603200.0,870115200.0
1981-08-20,INTC,1.991735,1.439910,1.746719,0.200678,-0.552163,-2.103513,-1.326483,-0.981572,-0.039683,0.003596,...,0.016077,0.016612,0.017475,0.032493,5577600.0,13920000.0,36988800.0,200044800.0,532262400.0,867436800.0
1981-08-21,INTC,-1.885045,0.456008,-0.403747,-0.736675,-1.184052,-2.147272,-1.537559,-1.093883,-0.016528,0.003025,...,0.015641,0.016884,0.017453,0.032511,8476800.0,17539200.0,41424000.0,201283200.0,526012800.0,867244800.0
1981-08-24,INTC,0.459583,0.045504,0.594965,-0.612037,-1.309778,-1.998272,-1.573805,-0.952140,-0.016805,0.003733,...,0.015060,0.017183,0.017383,0.032555,6441600.0,20496000.0,39782400.0,204278400.0,505363200.0,864796800.0
1981-08-25,INTC,-0.916318,-1.366914,-0.102815,0.106331,-1.319819,-1.772798,-1.708609,-0.965793,-0.025643,0.004159,...,0.014610,0.017508,0.017559,0.032611,13132800.0,28051200.0,47299200.0,207734400.0,507340800.0,840585600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-24,CRM,2.208215,0.090475,-0.525526,-0.990393,0.418297,1.150252,0.100756,-0.730036,-0.033887,5.870755,...,9.407252,10.298476,29.237392,39.151981,11438800.0,21698500.0,35853400.0,138261100.0,401663200.0,629483500.0
2022-08-25,CRM,-3.588170,-1.259835,-2.030746,-2.529826,-0.023651,0.917922,-0.289694,-1.100915,-0.049911,6.118278,...,9.278567,10.191764,28.953219,39.243692,24025600.0,40541200.0,56043100.0,156190600.0,407574200.0,648609600.0
2022-08-26,CRM,-1.231536,-2.039818,-2.404930,-2.244826,-0.495029,-0.306524,-0.581907,-1.158348,-0.030382,7.675522,...,9.142480,10.167686,28.602810,39.369134,11074700.0,46539100.0,63936500.0,161227600.0,381611500.0,653563900.0


In [23]:
# some eda of return
# df_norm_dropna_target.groupby(by="target_future_ret_24h_t")["target_future_ret_24h"].mean()